<a href="https://colab.research.google.com/github/peremartra/Large-Language-Model-Notebooks-Course/blob/main/4-Evaluating%20LLMs/4_3_evaluating_rag_giskard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
    <h1>Large Language Models Projects</a></h1>
    <h3>Apply and Implement Strategies for Large Language Models</h3>
    <h2>4.3-Evaluating LLMs with other LLMs.
  </h2>
    <h3>Evaluating a RAG solution with Giskard</h3>
</div>

by [Pere Martra](https://www.linkedin.com/in/pere-martra)

____________
Models: OpenAI.

Colab Environment: CPU-High RAM

Keys:
* LangChain.
* RAG.
* Giskard.
___________


In the final part of this notebook, you will find the necessary code to evaluate the suitability of the responses provided by the Agent using Giskard.

#Installing libraries & Loading Dataset

In [1]:
!pip install -q langchain==0.1.14
!pip install -q langchain-openai==0.1.1
!pip install langchainhub==0.1.15
#!pip install --upgrade -q tiktoken==0.5.2
!pip install -q datasets==2.16.1
!pip install -q chromadb==0.4.22
!pip install -q openai==1.30.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.

We will download the dataset from the Hugging Face datasets library. It's a dataset with information about diseases.

In [2]:
from datasets import load_dataset

data = load_dataset("keivalya/MedQuad-MedicalQnADataset", split='train')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:81: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
data = data.to_pandas()
data.head(10)

,qtype,Question,Answer
0,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,LCMV infections can occur after exposure to fr...
1,symptoms,What are the symptoms of Lymphocytic Choriomen...,LCMV is most commonly recognized as causing ne...
2,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,Individuals of all ages who come into contact ...
3,exams and tests,How to diagnose Lymphocytic Choriomeningitis (...,"During the first phase of the disease, the mos..."
4,treatment,What are the treatments for Lymphocytic Chorio...,"Aseptic meningitis, encephalitis, or meningoen..."
5,prevention,How to prevent Lymphocytic Choriomeningitis (L...,LCMV infection can be prevented by avoiding co...
6,information,What is (are) Parasites - Cysticercosis ?,Cysticercosis is an infection caused by the la...
7,susceptibility,Who is at risk for Parasites - Cysticercosis? ?,Cysticercosis is an infection caused by the la...
8,exams and tests,How to diagnose Parasites - Cysticercosis ?,"If you think that you may have cysticercosis, ..."
9,treatment,What are the treatments for Parasites - Cystic...,Some people with cysticercosis do not need to ...


In [4]:
data = data[0:100]

As you can see, the medical information in the dataset is well-organized, and to someone like me, who is not an expert in the field, it appears to be quite valuable. This information could be a useful addition to any general medicine book to support primary care doctors.

Load the langchain libraries to load the document.

In [5]:
from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores import Chroma

The Document is in the Answer column, and the others columns are Metadata.

In [6]:
df_loader = DataFrameLoader(data, page_content_column="Answer")


In [7]:
df_document = df_loader.load()
display(df_document[:2])

[Document(page_content='LCMV infections can occur after exposure to fresh urine, droppings, saliva, or nesting materials from infected rodents.  Transmission may also occur when these materials are directly introduced into broken skin, the nose, the eyes, or the mouth, or presumably, via the bite of an infected rodent. Person-to-person transmission has not been reported, with the exception of vertical transmission from infected mother to fetus, and rarely, through organ transplantation.', metadata={'qtype': 'susceptibility', 'Question': 'Who is at risk for Lymphocytic Choriomeningitis (LCM)? ?'}),
 Document(page_content='LCMV is most commonly recognized as causing neurological disease, as its name implies, though infection without symptoms or mild febrile illnesses are more common clinical manifestations. \n                \nFor infected persons who do become ill, onset of symptoms usually occurs 8-13 days after exposure to the virus as part of a biphasic febrile illness. This initial 

We can chunk the documents. The size to which we want to split the document is a design decision. The larger it is, the larger the prompt will be, and the slower the Model's response process.

We also need to consider the maximum prompt size and ensure that the document does not exceed it.

In [8]:
from langchain.text_splitter import CharacterTextSplitter

In [9]:
text_splitter = CharacterTextSplitter(chunk_size=1250, chunk_overlap=100)
texts = text_splitter.split_documents(df_document)


These warnings we see are because it can't perform the partition of the required size. This is because it waits for a page break to divide the text and does so when possible.

In [10]:
first_doc = texts[1]
print(first_doc.page_content)

LCMV is most commonly recognized as causing neurological disease, as its name implies, though infection without symptoms or mild febrile illnesses are more common clinical manifestations. 
                
For infected persons who do become ill, onset of symptoms usually occurs 8-13 days after exposure to the virus as part of a biphasic febrile illness. This initial phase, which may last as long as a week, typically begins with any or all of the following symptoms: fever, malaise, lack of appetite, muscle aches, headache, nausea, and vomiting. Other symptoms appearing less frequently include sore throat, cough, joint pain, chest pain, testicular pain, and parotid (salivary gland) pain. 
                
Following a few days of recovery, a second phase of illness may occur. Symptoms may consist of meningitis (fever, headache, stiff neck, etc.), encephalitis (drowsiness, confusion, sensory disturbances, and/or motor abnormalities, such as paralysis), or meningoencephalitis (inflammation 

### Initialize the Embedding Model and Vector DB

We load the text-embedding-ada-002 model from OpenAI.

In [11]:
from getpass import getpass
import os
if not 'OPENAI_API_KEY' in os.environ:
  os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key: ")

OpenAI API Key: ··········


In [12]:
from langchain_openai import OpenAIEmbeddings

model_name = 'text-embedding-ada-002'
#model_name = 'text-embedding-3-small'

embed = OpenAIEmbeddings(
    model=model_name,
    #openai_api_key=OPENAI_API_KEY
)

The execution of this cell may take 3 to 5 minutes. If you want it to be faster, you can reduce the number of records in the dataset.

In [13]:
directory_cdb = '/content/drive/MyDrive/chromadb'
chroma_db = Chroma.from_documents(
    df_document, embed, persist_directory=directory_cdb
)

We are going to create three objects.

* The language model, which can be any of those from OpenAI, the most common being gpt-3.5.
* The memory, responsible for keeping the prompt with all the necessary history.
* The retrieval, used to obtain information stored in ChromaDB.

In [14]:
from langchain.chat_models import ChatOpenAI
from langchain_openai import OpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA

llm=OpenAI(#openai_api_key=OPENAI_API_KEY,
           temperature=0.0)

conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=4, #Number of messages stored in memory
    return_messages=True #Must return the messages in the response.
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=chroma_db.as_retriever()
)

We can try the isolated Retrieval to see if the information it returns is relevant.




In [15]:
qa.run("What is the main symptom of LCM?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' The main symptom of LCM is a biphasic febrile illness, which includes symptoms such as fever, malaise, lack of appetite, muscle aches, headache, nausea, and vomiting.'

Perfect! The information returned is exactly what we desired.

## Creating the Agent.

In [16]:
from langchain.agents import Tool, AgentExecutor
DESCRIPTION = 'use this tool when answering medical knowledge queries to get more information about the topic'
#DESCRIPTION = 'Use this tool if any part of the QUERY or CHAT HISTORY history discusses concepts related to medicine or diseases to get more information about the topic'

#Defining the list of tool objects to be used by LangChain.
tools = [
    Tool(
        name='Medical KB',
        func=qa.run,
        description=(
            DESCRIPTION
        )
    )
]

In [17]:
from langchain.agents import initialize_agent, create_react_agent
from langchain import hub

prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(
    #agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    prompt=prompt,
    #verbose=True,
    #max_iterations=3,
    #early_stopping_method='generate'
)

In [18]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               verbose=True,
                               memory=conversational_memory,
                               max_iterations=30,
                               max_execution_time=600,
                               #early_stopping_method='generate',
                               handle_parsing_errors=True
                               )

### Using the Conversational Agent

To make queries we simply call the `agent` directly.

First i will try a order not related to the Medical field.

In [19]:
agent_executor.invoke({"input": "Give me the area of square of 2x2"})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Medical KB
Action Input: Area of square I don't know.Do I need to use a tool? No
Final Answer: The area of a square with sides of length 2 is 4.

> Finished chain.


{'input': 'Give me the area of square of 2x2',
 'chat_history': [],
 'output': 'The area of a square with sides of length 2 is 4.'}

Perfect, the model has responded without accessing the configured knowledge database.

Now I will try with a question that is also not related to health.

In [20]:
agent_executor.invoke({"input": "Do you know who is Clark Kent?"})



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
Final Answer: Clark Kent is the secret identity of the superhero Superman.

> Finished chain.


{'input': 'Do you know who is Clark Kent?',
 'chat_history': [HumanMessage(content='Give me the area of square of 2x2'),
  AIMessage(content='The area of a square with sides of length 2 is 4.')],
 'output': 'Clark Kent is the secret identity of the superhero Superman.'}

It has not accessed either, as the model has been able to identify that it is not a question related to the database that LangChain provides.

Now it's time to try with a question related to Medicine. Let's see if the model can understand that it should first look for information in the vector database at its disposal.

In [21]:
 agent_executor.memory.clear()

In [22]:
agent_executor.invoke({"input": """I have a patient that can have Botulism,
how can I confirm the diagnose?"""})



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Medical KB
Action Input: Botulism Botulism is a rare but serious paralytic illness caused by a nerve toxin produced by certain bacteria. It can be contracted through contaminated food, wounds, or ingestion of bacterial spores. Symptoms include muscle paralysis, difficulty swallowing, and respiratory failure. Treatment includes antitoxin, supportive care, and removal of the source of the toxin.Do I need to use a tool? No
Final Answer: To confirm the diagnosis, you can perform a physical exam and order laboratory tests, such as a stool or blood test, to detect the presence of the bacteria or its toxin. It is important to act quickly, as botulism can be life-threatening if left untreated.

> Finished chain.


{'input': 'I have a patient that can have Botulism,\nhow can I confirm the diagnose?',
 'chat_history': [],
 'output': 'To confirm the diagnosis, you can perform a physical exam and order laboratory tests, such as a stool or blood test, to detect the presence of the bacteria or its toxin. It is important to act quickly, as botulism can be life-threatening if left untreated.'}

Perfect, the most important thing for us is that it has been able to identify that it should go to the medical database to search for information about the symptoms.

In [23]:
agent_executor.invoke({"input": "Is this an important illness?"})



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
Final Answer: Yes, botulism is a serious illness that can be life-threatening if left untreated. It is important to seek medical attention and confirm the diagnosis as soon as possible.

> Finished chain.


{'input': 'Is this an important illness?',
 'chat_history': [HumanMessage(content='I have a patient that can have Botulism,\nhow can I confirm the diagnose?'),
  AIMessage(content='To confirm the diagnosis, you can perform a physical exam and order laboratory tests, such as a stool or blood test, to detect the presence of the bacteria or its toxin. It is important to act quickly, as botulism can be life-threatening if left untreated.')],
 'output': 'Yes, botulism is a serious illness that can be life-threatening if left untreated. It is important to seek medical attention and confirm the diagnosis as soon as possible.'}

And the memory works perfectly. We can maintain a conversation, taking into account that the model knows the previous questions and answers.

## Evaluating the solution with Giskard

Install and Load the libraries.

In [24]:
!pip install -q giskard[llm]==2.14.3
from giskard.rag import KnowledgeBase, generate_testset, evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 17.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.8/140.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.6/303.6 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85

Is necesary to create a Dataframe with just the column containing the information used to create the RAG system.

In [25]:
import pandas as pd
df_giskard = pd.DataFrame([d.page_content for d in df_document], columns=["text"])
df_giskard.head()

,text
0,LCMV infections can occur after exposure to fr...
1,LCMV is most commonly recognized as causing ne...
2,Individuals of all ages who come into contact ...
3,"During the first phase of the disease, the mos..."
4,"Aseptic meningitis, encephalitis, or meningoen..."


Using the information from the dataset, we ask Giskard to create a Knowledge Base, which is nothing more than a set of questions along with their respective answers. Both the questions and answers are generated by OpenAI's most advanced model, which is why it requires our OpenAI key to be provided.

In [26]:
kb_giskard = KnowledgeBase(df_giskard)

In [27]:
kb_giskard

In [40]:
# The more questions you generate, the more you will be charged.
test_questions = generate_testset(
    kb_giskard,
    num_questions=10,
    agent_description="Medical assistant for diagnosis and treatment support.",
)

Generating questions:   0%|          | 0/10 [00:00<?, ?it/s]

In [41]:
df_test_questions = test_questions.to_pandas()

In [42]:
df_test_questions.head()

,question,reference_answer,reference_context,conversation_history,metadata
id,,,,,
914cd13d-3cbd-464b-ab40-ecb3c58a3ecd,What are the guidelines provided in the context?,The context does not provide any specific guid...,Document 20: General guidelines for safe seafo...,[],"{'question_type': 'simple', 'seed_document_id'..."
e65de4a1-59f6-4842-b4e8-6d96370ef2c5,What are the recommended measures for preventi...,The recommended measures for preventing AHFV t...,Document 55: Transmission of AHFV is not well ...,[],"{'question_type': 'simple', 'seed_document_id'..."
aa4a7ecd-22be-4cbc-bd32-43afa9d74509,Could you describe the symptoms one might expe...,"Ocular larva migrans, when the larvae migrate ...",Document 68: If you suspect you have been infe...,[],"{'question_type': 'complex', 'seed_document_id..."
2d5d6ea8-e014-4c71-8bf5-470742acaf3c,What are some specific measures that individua...,"To prevent Babesiosis, people should avoid exp...",Document 75: Steps can be taken to reduce the ...,[],"{'question_type': 'complex', 'seed_document_id..."
548721bd-0909-4c06-a11a-9bf31a309654,Given that effective treatments are available ...,"To reduce the risk of Toxocara infection, have...",Document 41: Infected dogs and cats shed Toxoc...,[],"{'question_type': 'distracting element', 'seed..."


In [43]:
df_test_questions['question'].iloc[1]


'What are the recommended measures for preventing AHFV transmission?'

In [44]:
df_test_questions['reference_answer'].iloc[1]


'The recommended measures for preventing AHFV transmission are to avoid tick-infested areas and limit contact with livestock and domestic animals. Individuals should use tick repellants on skin and clothes, check skin for attached ticks, removing them as soon as possible. Tick collars are available for domestic animals, and dipping in acaricides is effective in killing ticks on livestock. People working with animals or animal products in farms or slaughterhouses should avoid unprotected contact with the blood, fluids, or tissues of any potentially infected or viremic animals.'

In [45]:
df_test_questions['reference_context'].iloc[1]

'Document 55: Transmission of AHFV is not well understood. AHFV is a zoonotic virus, and its described tick hosts (the soft tick Ornithodoros savignyi and the hard tick Hyalomma dromedari) are widely distributed. People can become infected through a tick bite or when crushing infected ticks. Epidemiologic studies indicate that contact with domestic animals or livestock may increase the risk of human infection. No human-to-human transmission of AHF has been documented. \n \nAlthough livestock animals may provide blood meals for ticks, it is thought that they play a minor role in transmitting AHFV to humans. No transmission through non-pasteurized milk has been described, although other tick-borne flaviviruses have been transmitted to humans through this route.\n\nDocument 60: Given that no treatment or specific prophylaxis is presently available, prevention and increased awareness of AHFV are the only recommended measures. Complete control of ticks and interruption of the virus life cyc

A function is created that will be called from Giskard's evaluate function. This function receives the question and returns the agent's response.

In [48]:
def use_agent(question, history=None):
  answer=agent_executor.invoke({"input": question})
  return answer['output']

In [49]:
agent_executor.memory.clear()

In [50]:
report = evaluate(use_agent, testset=test_questions, knowledge_base=kb_giskard)

Asking questions to the agent:   0%|          | 0/10 [00:00<?, ?it/s]



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
Final Answer: The guidelines provided in the context may vary depending on the specific topic or situation. Generally, guidelines are recommendations or rules that are meant to be followed in order to achieve a certain goal or outcome. They can be found in various contexts, such as in healthcare, education, or business. Some common guidelines include safety protocols, best practices, and ethical standards.

> Finished chain.


> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Medical KB
Action Input: AHFV transmission AHFV can be transmitted to humans through tick bites or by crushing infected ticks. Contact with domestic animals or livestock may also increase the risk of human infection. However, there is no evidence of human-to-human transmission of AHFV. Prevention measures include using tick repellants, avoiding tick-infested areas, and limiting contact with livestock an

CorrectnessMetric evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

In [51]:
# Summary with the results.
report.correctness_by_question_type()

,correctness
question_type,
complex,1.0
conversational,0.0
distracting element,0.5
double,1.0
simple,0.5
situational,1.0


In [52]:
report.correctness_by_topic()

,correctness
topic,
Infectious Diseases Diagnosis and Transmission,1.000000
Others,0.000000
Seafood Poisoning and Botulism,0.333333


In [53]:
report.component_scores()

,score
RAG Components,
GENERATOR,0.8
RETRIEVER,0.5
REWRITER,0.5
ROUTING,1.0
KNOWLEDGE_BASE,0.0


In [54]:
# Obtaining the incorrect answers
failures = report.get_failures()[:2]
failures

,question,reference_answer,reference_context,conversation_history,metadata,agent_answer,correctness,correctness_reason
id,,,,,,,,
914cd13d-3cbd-464b-ab40-ecb3c58a3ecd,What are the guidelines provided in the context?,The context does not provide any specific guid...,Document 20: General guidelines for safe seafo...,[],"{'question_type': 'simple', 'seed_document_id'...",The guidelines provided in the context may var...,False,The agent's answer is incorrect because it doe...
8e066850-f6a2-410f-8567-9769bcee17c4,In the context of managing marine toxin poison...,The Centers for Disease Control and Prevention...,Document 21: Some health departments test shel...,[],"{'question_type': 'distracting element', 'seed...",The CDC plays a crucial role in managing both ...,False,The agent's answer includes information about ...


In [55]:
# Giskard explains the reasons why it considers the answers to be incorrect.
failures['correctness_reason'].iloc[1]

"The agent's answer includes information about the CDC's role in managing rabies exposure, which is not mentioned in the ground truth. The ground truth only refers to the CDC's role in managing marine toxin poisoning."


# Conclusions.
The experiment has been a small success. The Vectorial database has been configured and filled with information from the dataset. A LangChain agent has been created, and it has been able to retrieve information from the database only when necessary. Don't forget that our ChatBot has memory.

All of this in just a few lines of code!


---